<a href="https://colab.research.google.com/github/raamav/Text-Classification/blob/master/Twitter_for_PublicEmergencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "Real or NOT" : NLP to Identify Public Emergency Related Tweets


## Introduction

Being able to predict which Tweets are about real Public Emergencies (*eg Earthquakes, Floods, Terrorist Events*) and which ones are not. 

(The words 'Pubic Emergency' and 'Disasters' have been used interchangeably)

> Twitter has become an important communication channel in times of emergency.
The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

> But, it’s not always clear whether a person’s words are actually announcing a disaster. 

*Source: [Real or Not? NLP with Disaster Tweets, A Kaggle Competition](https://www.kaggle.com/c/nlp-getting-started/overview)*

<BR>

### Goal

This is the second exploration of the dataset. In the 1st attempt an ensemble of ensemble methods yielded 80% accuracy on the test data. 

The goal is to get a better model using Deep Learning.

<BR>

### Approach Summary
*TBD*

<BR>

### Results
*tbd*

<BR>

***


## Exploring an LSTM Based Model (w/Pre-Trained Embeddings)

In [1]:
# STANDARD IMPORTS

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

tf.__version__


'2.2.0-rc3'

In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/raamav/Text-Classification/master/train.csv")

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


One of the first steps is to divide the data into training, validation and test sets. Will be adopting a 70:15:15 split for this purpose

In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size = 0.3, random_state = 42)
val, test = train_test_split(test, test_size = 0.5, random_state = 42)

In [14]:
print("train.shape", train.shape)
print("val.shape", val.shape)
print("test.shape", test.shape)

train.shape (5329, 5)
val.shape (1142, 5)
test.shape (1142, 5)


Exploring the data, building functions (and/or pipelines) for preprocessing data. Will also explore the possibility of crating features (although deep learning models do not require extensive feature engineering)